# Session 3 - Parameter management

In session 1 we saw how to choose a parameter set from the ones built in PyBaMM, and how to change some of these parameters. In this notebook we will see some advanced features in parameter management.

We start by importing PyBaMM:

In [1]:
%pip install pybamm -q    # install PyBaMM if not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


## How can I define a new parameter set?

Manually changing the parameters works very well when we need to change a few parameters, but it is not ideal when we have to change the entire parameter set (especially if we want to run many different scripts with the same new parameter set). In these situations, it is better to define a new parameter set.

Default parameter sets in PyBaMM are located inside your installation directory, typically something like
```bash
../venv/lib/pythonX.Y/site-packages/pybamm/input/parameters/
```

The `input/parameters` directory has the following structure:
```
input/parameters/
  lithium_ion/ # Chemistry
    negative_electrodes/ # Component
      graphite_Chen2020/ # Parameter set
        parameters.csv
        graphite_LGM50_diffusivity_Chen2020.py
        ...
      graphite_mcmb2528_Marquis2019/
      graphite_Ecker2015/
      ...
    positive_electrodes/
    cells/
    electrolytes/
    seis/
    separators/
    experiments/
  lead_acid/
    ...
```

Note that this matches the `chemistry` defined for a parameter set (see Session 1 for more details):

In [2]:
pybamm.parameter_sets.Chen2020

{'chemistry': 'lithium_ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'graphite_Chen2020',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

The chemistry (`Chen2020` here) is defined from different components (e.g. `cell`, `electrolyte`...) and a parameter set is associated to each component (e.g. `LGM50_Chen2020`, `lipf6_Nyman2008`...). The parameter set of each component matches the name of a subdirectory in `input/parameters`.

When setting parameter values using `pybamm.ParameterValues` and a filename or chemistry, the file is actually searched in several locations. These locations are specified by `pybamm.PARAMETER_PATH`. The default is

In [3]:
pybamm.PARAMETER_PATH

['/home/ferranbrosa/workshop-notebooks/env/lib/python3.7/site-packages',
 '/home/ferranbrosa/workshop-notebooks/2021-09-workshop/Session_3',
 '/home/ferranbrosa/workshop-notebooks/env/lib/python3.7/site-packages/pybamm/input/parameters']

so the parameter files will first be searched in the root directory, then the current directory and finally where the `input/parameters` directory within the PyBaMM installation directory (where the default parameter are placed). You can also add a search location by modifying the `PARAMETER_PATH` list.

**NB:** This means that if a parameter set is in the current directory and has the same name as a default one it will override the values of the default parameter set, which we use in the next section.

### Defining a new parameter set

Quite often it is easier to define new parameter sets by editing existing ones. We strongly recommend to not alter the default parameter sets in `input/parameters`. Instead, we can pull the parameter sets for a given chemistry into the current directory. Then, any changes on those parameters will override the default ones.

To do so, we need to run some bash code. However, we can do this directly in the Juptyer notebook by starting the cell with the tag `%%bash`.

In [4]:
%%bash
pybamm_edit_parameter lithium_ion

The above command creates a `lithium_ion` directory in the current directory, populated with copies of the default parameters that we can edit. Note that if a file called `lithium_ion` already exists, maybe because you already executed the command, you will get an error when you run the cell above.

If we take a look inside `lithium_ion` we see that the directory is structured into components as we explained earlier.

In [5]:
%%bash
ls lithium_ion

__init__.py
__pycache__
cells
electrolytes
experiments
lithium_platings
negative_electrodes
positive_electrodes
seis
separators


We could modify any of the existing parameter sets without worrying of messing up with PyBaMM defaults, but here we will define a new parameter set. As an example, we will create a new negative electrode parameter set starting from `graphite_Chen2020`. To do so, we create a copy of the parameter set and we call it `graphite_workshop`

In [6]:
%%bash
cp -r lithium_ion/negative_electrodes/graphite_Chen2020/. lithium_ion/negative_electrodes/graphite_workshop

Now we can go and modify the parameters with your favourite text editor.

To run simulations with the new parameter set we can import an existing `chemistry` and modify one of the components.

In [7]:
chemistry = pybamm.parameter_sets.Chen2020
chemistry["negative electrode"] = "graphite_workshop"

So if we check the chemistry we see that all the components are the default for `Chen2020` except for the negative electrode which is the new one we defined:

In [8]:
chemistry

{'chemistry': 'lithium_ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'graphite_workshop',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

Now we can import the parameter for this specific chemistry and run the simulations as usual

In [9]:
parameter_values = pybamm.ParameterValues(chemistry=chemistry)
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3553.12977501657, step=35.5312977501657), Ou…

### Contributing a new parameter set to PyBaMM

Once you're happy with a particular parameter set of your creation, you may want to contribute it to the PyBaMM parameter distribution, so that it comes bundled with future release of PyBaMM. This will require you to open a Pull Request on the PyBaMM repository, a process that is described here.

A prerequisite is that you add you parameter set to the default parameter directory. You could manually copy the corresponding directory to your installation directory, but the command `pybamm_add_param` can do it for you:

In [10]:
%%bash
# pybamm_add_parameter <dir> <chemistry> <component>
pybamm_add_parameter lithium_ion/negative_electrodes/graphite_workshop lithium_ion negative_electrodes

To test that the new parameter set has been added to the default ones, we remove it from the 

In [11]:
%%bash
rm -r lithium_ion/negative_electrodes/graphite_workshop

Then we can define again the same chemistry as before and run the simulation.

In [12]:
chemistry = pybamm.parameter_sets.Chen2020
chemistry["negative electrode"] = "graphite_workshop"
parameter_values = pybamm.ParameterValues(chemistry=chemistry)
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3553.12977501657, step=35.5312977501657), Ou…

If you think you made a mistake when adding the parameters, you can always delete a specific parameter set using `pybamm_rm_param`.

In [13]:
%%bash
pybamm_rm_parameter -f lithium_ion/negative_electrodes/graphite_workshop lithium_ion negative_electrodes

Before finishing, let's remove the local `lithium_ion` directory to leave this directory as we found it

In [14]:
%%bash
rm -rf lithium_ion